In [ ]:
import pandas as pd
from pathlib import Path
import pickle
from sklearn.metrics import roc_auc_score
from kaggle import api
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

In [ ]:
path=Path('./data')

In [ ]:
# train_p1 = pd.read_feather(path/'train_p1_transformed.feather')
train_p2 = pd.read_feather(path/'train_p2_transformed.feather')
val_p1 = pd.read_feather(path/'val_p1_transformed.feather')
# val_p2 = pd.read_feather(path/'val_p2_transformed.feather')

In [ ]:
train_p2.shape, val_p1.shape, val_p2.shape

In [ ]:
train_p2 = train_p2.drop(index=train_p2[train_p2['content_type_id']==1].index, columns = ['content_type_id']).reset_index(drop=True)
val_p1 = val_p1.drop(index=val_p1[val_p1['content_type_id']==1].index, columns = ['content_type_id']).reset_index(drop=True)
# val_p2 = val_p2.drop(index=val_p2[val_p2['content_type_id']==1].index, columns = ['content_type_id']).reset_index(drop=True)

In [ ]:
train_p2.shape, val_p1.shape, #val_p2.shape

In [ ]:
train_p2_val_p1 = train_p2.append(val_p1).reset_index(drop=True)

In [ ]:
train_p2_val_p1.shape

In [ ]:
target = 'answered_correctly'
# features = list(train_p2_val_p1.columns)
features = list(val_p1.columns)
# features.remove('user_id')
features.remove(target)

In [ ]:
print(features)

In [ ]:
cont_nn = ['timestamp', 'task_container_id', 'prior_question_elapsed_time',
           'question_mean', 'question_std', 'question_skew', 'time_between',
           'question_cumcount', 'prior_question_elapsed_time_cumsum',
           'time_per_question', 'timestamp_prior_time_cumsum_diff']

cat_nn = ['user_id', 'content_id', 'prior_question_had_explanation', 'bundle_id',
          'part', 'num_of_tags', 'bundle_size']

In [ ]:
train_idx = train_p2_val_p1[:len(train_p2)].index
val_idx = train_p2_val_p1[len(train_p2):].index

In [ ]:
splits = (list(train_idx),list(val_idx))

In [ ]:
procs_nn = [Categorify,
            FillMissing,
            Normalize]

In [ ]:
to_nn = TabularPandas(
    df = train_p2_val_p1,
    procs=procs_nn,
    cat_names=cat_nn,
    cont_names=cont_nn,
    y_names=target,
    y_block=None,
    splits=splits,
    do_setup=True,
    device=None,
    inplace=False,
    reduce_memory=True,
)

In [ ]:
train_p2_pred = TabularPandas(train_p2, procs_nn, cat_nn, cont_nn,
                      splits=None, y_names=target)

In [ ]:
val_p1_pred = TabularPandas(val_p1, procs_nn, cat_nn, cont_nn,
                      splits=None, y_names=target)
val_p2_pred = TabularPandas(val_p2, procs_nn, cat_nn, cont_nn,
                      splits=None, y_names=target)

In [ ]:
dls = to_nn.dataloaders(4096)
# dls = train_p2_pred.dataloaders(1024)

In [ ]:
learn = tabular_learner(dls, layers=[512,256], y_range=[0,1],
                        n_out=None, loss_func=BCELossFlat(), model_dir=path/'models', metrics=accuracy)

In [ ]:
learn.load('nn-t_p2-768-256-ts-before-e2')

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(1, 0.003)

In [ ]:
learn.fit_one_cycle(1, 0.001)

In [ ]:
learn.save('nn-t_p2-768-256-new-e1')

In [ ]:
def predict_batch(self, df):
    dl = self.dls.test_dl(df)
    dl.dataset.conts = dl.dataset.conts.astype(np.float32)
    inp,preds,_,dec_preds = self.get_preds(dl=dl, with_input=True, with_decoded=True)
    return preds.numpy()

setattr(learn, 'predict_batch', predict_batch)

In [ ]:
preds_train_p2 = learn.predict_batch(learn, train_p2[features])
preds_val_p1 = learn.predict_batch(learn, val_p1[features])
preds_val_p2 = learn.predict_batch(learn, val_p2[features])

In [ ]:
(preds_val_p2[-5:], val_p2[dep_var][-5:])

In [ ]:
# 1 epochs lr 0.0005
(roc_auc_score(train_p2[dep_var], preds_train_p2),
 roc_auc_score(val_p1[dep_var], preds_val_p1),
 roc_auc_score(val_p2[dep_var], preds_val_p2))

In [ ]:
# 2 epochs lr 0.0005
(roc_auc_score(train_p2[dep_var], preds_train_p2),
 roc_auc_score(val_p1[dep_var], preds_val_p1),
 roc_auc_score(val_p2[dep_var], preds_val_p2))

In [ ]:
# 3 epochs lr 0.0005
(roc_auc_score(train_p2[dep_var], preds_train_p2),
 roc_auc_score(val_p1[dep_var], preds_val_p1),
 roc_auc_score(val_p2[dep_var], preds_val_p2))

In [ ]:
# 4 epochs lr 0.0005
(roc_auc_score(train_p2[dep_var], preds_train_p2),
 roc_auc_score(val_p1[dep_var], preds_val_p1),
 roc_auc_score(val_p2[dep_var], preds_val_p2))

In [ ]:
# 5 epochs lr 0.0005
(roc_auc_score(train_p2[dep_var], preds_train_p2),
 roc_auc_score(val_p1[dep_var], preds_val_p1),
 roc_auc_score(val_p2[dep_var], preds_val_p2))

In [ ]:
# 1 epochs lr 0.0005 1024-512-256
(roc_auc_score(train_p2[dep_var], preds_train_p2),
 roc_auc_score(val_p1[dep_var], preds_val_p1),
 roc_auc_score(val_p2[dep_var], preds_val_p2))

In [ ]:
# 1 epochs lr 0.001 ts-before
(roc_auc_score(train_p2[dep_var], preds_train_p2),
 roc_auc_score(val_p1[dep_var], preds_val_p1),
 roc_auc_score(val_p2[dep_var], preds_val_p2))

In [ ]:
# 2 epochs lr 0.001 ts-before
(roc_auc_score(train_p2[dep_var], preds_train_p2),
 roc_auc_score(val_p1[dep_var], preds_val_p1),
 roc_auc_score(val_p2[dep_var], preds_val_p2))

In [ ]:
# 3 epochs lr 0.0005 1024-512-256-ts-before
(roc_auc_score(train_p2[dep_var], preds_train_p2),
 roc_auc_score(val_p1[dep_var], preds_val_p1),
 roc_auc_score(val_p2[dep_var], preds_val_p2))

In [ ]:
# 1 epochs lr 0.003 768-256-new
(roc_auc_score(train_p2[dep_var], preds_train_p2),
 roc_auc_score(val_p1[val_p1['content_type_id'] == 0][dep_var], preds_val_p1),
 roc_auc_score(val_p2[val_p2['content_type_id'] == 0][dep_var], preds_val_p2))

In [ ]:
# 2 epochs lr 0.002 768-256-new
(roc_auc_score(train_p2[dep_var], preds_train_p2),
 roc_auc_score(val_p1[dep_var], preds_val_p1),
 roc_auc_score(val_p2[dep_var], preds_val_p2))

In [ ]:
train_p2.columns

In [ ]:
train_p2['diff'] = train_p2['timestamp'] - train_p2['prior_question_elapsed_time_cumsum']
val_p1['diff'] = val_p1['timestamp'] - val_p1['prior_question_elapsed_time_cumsum']
val_p2['diff'] = val_p2['timestamp'] - val_p2['prior_question_elapsed_time_cumsum']

In [ ]:
target = 'answered_correctly'
features = list(val_p1.columns)
features.remove(target)
features.remove('user_id')
# features.remove('timestamp')
# features.remove('task_container_id')
# features.remove('question_mean')
# features.remove('question_std')
# features.remove('question_skew')

In [ ]:
train_p2.shape

In [ ]:
print(features)

In [ ]:
cat = ['content_id', 'task_container_id', 'prior_question_had_explanation',
       'bundle_id', 'part', 'num_of_tags', 'bundle_size',
       'question_cumcount', 'is_prior_grater']

RandomForest Classifier

In [ ]:
rf = RandomForestClassifier(n_estimators=60,
    criterion='gini',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=1.0,
    max_leaf_nodes=31, #default - None
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    bootstrap=True,
    oob_score=False,
    n_jobs=-1,
    random_state=37,
    verbose=1,
    warm_start=False,
    class_weight=None,
    ccp_alpha=0.0,
    max_samples=0.01)

In [ ]:
rf.fit(train_p2[features].values, train_p2[target].values)

In [ ]:
rf_preds_train_p2 = rf.predict_proba(train_p2[features])[:,1]
rf_preds_p1 = rf.predict_proba(val_p1[features])[:,1]
rf_preds_p2 = rf.predict_proba(val_p2[features])[:,1]

In [ ]:
rf_preds_p2[-5:]

In [ ]:
# rf 40 estimators (new)
(roc_auc_score(train_p2[target], rf_preds_train_p2),
 roc_auc_score(val_p1[target], rf_preds_p1),
 roc_auc_score(val_p2[target], rf_preds_p2))

In [ ]:
# rf 60 estimators (new)
(roc_auc_score(train_p2[target], rf_preds_train_p2),
 roc_auc_score(val_p1[target], rf_preds_p1),
 roc_auc_score(val_p2[target], rf_preds_p2))

In [ ]:
# rf 60 estimators (new)
(roc_auc_score(train_p2[target], rf_preds_train_p2),
 roc_auc_score(val_p1[target], rf_preds_p1),
 roc_auc_score(val_p2[target], rf_preds_p2))

In [ ]:
fi = rf.feature_importances_

In [ ]:
fi[fi.argsort()]

In [ ]:
pd.DataFrame(zip(features, fi), columns=['features', 'importance']).sort_values(by='importance', ascending=False)

In [ ]:
preds_estimators = np.stack([t.predict_proba(val_p1[features])[:,1] for t in rf.estimators_])

In [ ]:
plt.plot([roc_auc_score(val_p1[target], preds_estimators[:i+1].mean(0)) for i in range(len(rf.estimators_))]);

XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
print(dir(xgb))

In [ ]:
dtrain_p2 = xgb.DMatrix(data=train_p2[features], label=train_p2[target], weight=None, base_margin=None,
                        missing=None, silent=True, feature_names=features,
                        feature_types=None, nthread=-1,)

In [ ]:
dval_p1 = xgb.DMatrix(data=val_p1[features], label=val_p1[target], weight=None, base_margin=None,
                        missing=None, silent=True, feature_names=features,
                        feature_types=None, nthread=-1,)

In [ ]:
dval_p2 = xgb.DMatrix(data=val_p2[features], label=val_p2[target], weight=None, base_margin=None,
                        missing=None, silent=True, feature_names=features,
                        feature_types=None, nthread=-1,)

In [ ]:
params = {
    # Parameters for Tree Booster
    'learning_rate':0.6, 'min_split_loss':0,
    'max_depth':3, 'subsample':0.1, 'colsample_bytree':1.0,
    
    # Learning Task Parameters
    'objective':'binary:logistic',
    'eval_metric':'auc'
}

In [ ]:
xgb_booster=xgb.train(params=params, dtrain=dtrain_p2, num_boost_round=30,
          evals=((dtrain_p2, 'train_p2'), (dval_p1, 'val_p1'), (dval_p2, 'val_p2')),
          obj=None, feval=None, maximize=False,
          early_stopping_rounds=None, evals_result=None,
          verbose_eval=5, xgb_model=None, callbacks=None)

In [ ]:
xgb.plot_importance(xgb_booster, importance_type='weight')

LightGBM

In [ ]:
import lightgbm as lgb

In [ ]:
print(dir(lgb))

In [ ]:
# gtrain = lgb.Dataset(data=train_df[features], label=train_df[target], reference=None,
#                         weight=None, group=None, init_score=None,
#                         silent=False, feature_name='auto',
#                         categorical_feature='auto', params=None, free_raw_data=True)

In [ ]:
# gval = lgb.Dataset(data=val_df[features], label=val_df[target], reference=gtrain,
#                         weight=None, group=None, init_score=None,
#                         silent=False, feature_name='auto',
#                         categorical_feature='auto', params=None, free_raw_data=True)

In [ ]:
gtrain_p2 = lgb.Dataset(data=train_p2[features], label=train_p2[target], reference=None,
                        weight=None, group=None, init_score=None,
                        silent=False, feature_name='auto',
                        categorical_feature='auto', params=None, free_raw_data=True)

In [ ]:
gval_p1 = lgb.Dataset(data=val_p1[features], label=val_p1[target], reference=gtrain_p2,
                        weight=None, group=None, init_score=None,
                        silent=False, feature_name='auto',
                        categorical_feature='auto', params=None, free_raw_data=True)

In [ ]:
gval_p2 = lgb.Dataset(data=val_p2[features], label=val_p2[target], reference=gtrain_p2,
                        weight=None, group=None, init_score=None,
                        silent=False, feature_name='auto',
                        categorical_feature='auto', params=None, free_raw_data=True)

In [ ]:
lgb_params={
    #Core Parameters
    'objective':'binary', #cross_entropy
    'learning_rate':0.6,
    'num_leaves':31, #31
    'num_threads':4,
    'device_type':'cpu',
    
    #Learning Control Parameters
    'max_depth':None,
#     'feature_pre_filter':False, # to change the min_data_in_leaf
    'min_data_in_leaf':20, # 20
    'bagging_fraction':0.05,
    'feature_fraction':1.0,
    
    #Metric Parameters
    'metric':'auc'
}

In [ ]:
# lgb_booster = lgb.train(params=lgb_params, train_set=gtrain, num_boost_round=50,
#                         valid_sets=[gtrain, gval], valid_names=['train_df', 'val_df'],
#                         fobj=None, feval=None,
#                         init_model=None, feature_name='auto', categorical_feature='auto',
#                         early_stopping_rounds=None, evals_result=None, verbose_eval=5,
#                         learning_rates=None, keep_training_booster=False, callbacks=None)

In [ ]:
lgb_booster = lgb.train(params=lgb_params, train_set=gtrain_p2, num_boost_round=50,
                        valid_sets=[gtrain_p2, gval_p1, gval_p2], valid_names=['train_p2','val_p1', 'val_p2'],
                        fobj=None, feval=None,
                        init_model=None, feature_name='auto', categorical_feature='auto',
                        early_stopping_rounds=None, evals_result=None, verbose_eval=5,
                        learning_rates=None, keep_training_booster=False, callbacks=None)

In [ ]:
lgb_booster.save_model(path.stem+'/models/lgb_p2_50')

In [ ]:
lgb.plot_importance(lgb_booster)

In [ ]:
xgb_preds_p1 = xgb_booster.predict(dval_p1)
xgb_preds_p2 = xgb_booster.predict(dval_p2)

In [ ]:
xgb_preds_p2

In [ ]:
lgb_preds_p1 = lgb_booster.predict(val_p1[features])
lgb_preds_p2 = lgb_booster.predict(val_p2[features])

In [ ]:
lgb_preds_p2

In [ ]:
print("xgb --> val_p1: {:.4}, val_p2: {:.4}".format(
    roc_auc_score(val_p1[dep_var], xgb_preds_p1),
    roc_auc_score(val_p2[dep_var], xgb_preds_p2)))

In [ ]:
print("lbm --> val_p1: {:.4}, val_p2: {:.4}".format(
    roc_auc_score(val_p1[dep_var], lgb_preds_p1),
    roc_auc_score(val_p2[dep_var], lgb_preds_p2)))

In [ ]:
print("rf --> val_p1: {:.4}, val_p2: {:.4}".format(
    roc_auc_score(val_p1[dep_var], rf_preds_p1),
    roc_auc_score(val_p2[dep_var], rf_preds_p2)))

In [ ]:
w = 0.1
avg_preds_p1 = xgb_preds_p1*w + lgb_preds_p1*(1-w)
avg_preds_p2 = xgb_preds_p2*w + lgb_preds_p2*(1-w)
print("avg (xgb:{}) --> val_p1: {:.4}, val_p2: {:.4}".format(
    w,
    roc_auc_score(val_p1[dep_var], avg_preds_p1),
    roc_auc_score(val_p2[dep_var], avg_preds_p2)))

In [ ]:
w = 0.2
avg_preds_p1 = xgb_preds_p1*w + lgb_preds_p1*(1-w)
avg_preds_p2 = xgb_preds_p2*w + lgb_preds_p2*(1-w)
print("avg (xgb:{}) --> val_p1: {:.4}, val_p2: {:.4}".format(
    w,
    roc_auc_score(val_p1[dep_var], avg_preds_p1),
    roc_auc_score(val_p2[dep_var], avg_preds_p2)))

In [ ]:
w = 0.3
avg_preds_p1 = xgb_preds_p1*w + lgb_preds_p1*(1-w)
avg_preds_p2 = xgb_preds_p2*w + lgb_preds_p2*(1-w)
print("avg (xgb:{}) --> val_p1: {:.4}, val_p2: {:.4}".format(
    w,
    roc_auc_score(val_p1[dep_var], avg_preds_p1),
    roc_auc_score(val_p2[dep_var], avg_preds_p2)))

In [ ]:
w = 0.4
avg_preds_p1 = xgb_preds_p1*w + lgb_preds_p1*(1-w)
avg_preds_p2 = xgb_preds_p2*w + lgb_preds_p2*(1-w)
print("avg (xgb:{}) --> val_p1: {:.4}, val_p2: {:.4}".format(
    w,
    roc_auc_score(val_p1[dep_var], avg_preds_p1),
    roc_auc_score(val_p2[dep_var], avg_preds_p2)))

In [ ]:
w = 0.5
avg_preds_p1 = xgb_preds_p1*w + lgb_preds_p1*(1-w)
avg_preds_p2 = xgb_preds_p2*w + lgb_preds_p2*(1-w)
print("avg (xgb:{}) --> val_p1: {:.4}, val_p2: {:.4}".format(
    w,
    roc_auc_score(val_p1[dep_var], avg_preds_p1),
    roc_auc_score(val_p2[dep_var], avg_preds_p2)))

In [ ]:
w = 0.6
avg_preds_p1 = xgb_preds_p1*w + lgb_preds_p1*(1-w)
avg_preds_p2 = xgb_preds_p2*w + lgb_preds_p2*(1-w)
print("avg (xgb:{}) --> val_p1: {:.4}, val_p2: {:.4}".format(
    w,
    roc_auc_score(val_p1[dep_var], avg_preds_p1),
    roc_auc_score(val_p2[dep_var], avg_preds_p2)))

In [ ]:
train_p1

In [ ]:
train_p1[train_p1['user_id']==13134].tail(3)

In [ ]:
train_p2[train_p2['user_id']==13134].tail(3)

In [ ]:
val_p1[val_p1['user_id']==13134].tail(3)

In [ ]:
val_p2[val_p2['user_id']==13134].head(3)